In [2]:
import numpy as np
import math
import itertools
import picos as pic

In [9]:
def SDP(n,K,d,p,D):
    
    def Swap_multiplication(A,B):

        C = tuple(set(A)^set(B))

        return C 


    def K_fun(M,K,N):

        Fun=0

        for q in range(0,M+1):
            Fun+= math.comb(N-K,M-q)*math.comb(K,q)*(-1)**q
        return Fun 

    T=2**p ### Total number of reductions ###

    Generators=list(range(0, p))

    Elements=[]

    for i in range(n+1):
        Elements += (list(itertools.combinations(Generators, i)))


    Elements_R={new_label:old_label for old_label, new_label in enumerate(Elements)}

    rho=[]

    length=0

    for i in range(0,d):

            length+=math.comb(p,i)
            
    length_c=0

    for i in range(d,n+1):

            length_c+=math.comb(p,i)

    Rep=[]

    for i in range(T):
        for j in range(T):
            C=Swap_multiplication(Elements[i],Elements[j])
            Rep.append([C,i,j])  

    Rep=sorted(Rep)


    ##### SDP #####


    pic.available_solvers()

    F = pic.SymmetricVariable('F', (T,T)) # the moment matrix

    M=np.zeros((T, T))

    sdp = pic.Problem()

    sdp.set_objective('find') 



    ##### Adding state constraints #####

    sdp.add_constraint(F[0,0]==K**2)
    sdp.add_constraint(F[0,T-1]==K)
    
    
    ##### Adding K-L constrains #####
    
    for i in range(0,length):

        S_c=Swap_multiplication(Elements[i],Elements[T-1])

        sdp.add_constraint(K*F[0,Elements_R[S_c]]-F[0,i]==0)
    
    
    for i in range(0,length_c):

        S_c=Swap_multiplication(Elements[i],Elements[T-1])

        sdp.add_constraint(K*F[0,Elements_R[S_c]]-F[0,i]>=0)
        
    
    
    ##### Adding A_j constrains #####
    
    for m in range(p+1):
        sdp.add_constraint(sum([(-1)**(m-j)*D**(j)*math.comb(n-j,n-m)*F[0,Elements_R[comb]] for j in range(m+1) 
                                 for comb in itertools.combinations(Generators, j)])  >> 0)
    """
    
    sdp.add_list_of_constraints([F[0,Elements_R[comb]] >= D**(-i)*K**2 for i in range(p+1) 
                             for comb in itertools.combinations(Generators, i)])
    """
    ##### Adding Shadow constrains #####

    
    for j in range(p+1):
        sdp.add_constraint(sum([K_fun(p-j,i,p)*F[Elements_R[comb],0]
                                    for i in range(p+1) for comb in itertools.combinations(Generators, i)]) >= 0)
    
    
    
    ##### Adding NPA constrains #####

    sdp.add_constraint(F>>0)

    for rep in Rep:
        if rep==Rep[0]:  

            rep_old=rep.copy()

        else:

            if set(rep[0])==set(rep_old[0]):
                
                sdp.add_constraint(F[rep[1],rep[2]]==F[rep_old[1],rep_old[2]])
            else:
                rep_old=rep.copy() 


    sol = sdp.solve(solver='mosek', primals=False)
    
    #print(np.round(np.array(F),2))
    
    if sol.problemStatus == "feasible":
        return True                    
    elif sol.problemStatus == "infeasible":
        return False
    else:
        return print("WTF") 

    
D=2

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],
    
[8,0,4],[8,1,3],[8,2,3],[8,3,3],[8,4,2],[8,5,2],[8,6,2],[8,7,1],[8,8,1],

[9,0,4],[9,1,3],[9,2,3],[9,3,3],[9,4,2],[9,5,2],[9,6,2],[9,7,1],[9,8,1],[9,9,1],

[10,0,4],[10,1,4],[10,2,4],[10,3,3],[10,4,3],[10,5,2],[10,6,2],[10,7,2],[10,8,2],[10,9,1],[10,10,1]
]

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1]]


#### Existing qubit codes ####

for code in Codes:
    
    n,K,d=code
    p=n
    check=SDP(n,2**K,d,p,D)
    
    if check != True:
        print("%s=%s" % (code,check))

print("Done")



#### Not existing qubit codes (distance+1) ####

for code in Codes:
    
    n,K,d=code
    p=n
    check=SDP(n,2**K,d+1,p,D)
    
    if check != False:
        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))

print("Done")


Done
[1,0,2]=True
[2,0,3]=True
[3,0,3]=True
[4,0,3]=True
[5,0,4]=True
[6,0,5]=True
[7,0,4]=True
Done


In [ ]:

D=2 ### Dimension ###

n=9 ### n of qudits ###

p=9 ### Partitions ###

K=112 ### Encoding ###

d=2 ### Distance of a code ###

    
    
print("n=%s" % SDP(n,K,d,p,D))
print("\n")

D=2 ### Dimension ###

n=7 ### n of qudits ###

p=9 ### Partitions ###

K=113 ### Encoding ###

d=2 ### Distance of a code ###

    
    
print("n=%s" % SDP(n,K,d,p,D))
print("\n")

In [ ]:

D=2 ### Dimension ###

n=7 ### n of qudits ###

K=11 ### Encoding ###

d=4 ### Distance of a code ###

p=7 ### Partitions ###
    
for i in range(1,2):
    print("%s=%s" % (i,SDP(n,i,d,p,D)))